## WOMANIUM 2024
#### Classiq Module 6.II 
#### Homework 6.12
#### Francini Corrales Garro

### Instructions
#### Final assignment 

Follow the example from Bootcamp 4 Links to an external site. for creating the quantum walk operator for the case of a circle with 4 nodes, and design the quantum walk operator for the case of a line with 16 nodes:

Classiq_FinalAssignment_Fig1.png

A. Create a well-detailed Python Jupyter notebook that explains your algorithm, including the code parts covered in class, and pictures/figures where relevant.

Utilize the Python code from class: quantum_walk_circle_example.pyLinks to an external site.. It can be found directly also in the Classiq Git Library in the community/womanium/assignmentsLinks to an external site. folder.
Feel free to extend the example beyond the requirements here and what was covered in class.

B. Contribute your notebook to the Classiq Git LibraryLinks to an external site. to the folder community/womanium/assignmentsLinks to an external site.

Follow the contribution guidelinesLinks to an external site.  in order to contribute - NO need to open an issue for this, you can directly open a PR
The PR title should be: Womanium Final Assignment - Francini Corrales Garro

The file name should be in the following format: Francini_Corrales_Garro_hw4.ipynb

# Explanation 

The following image illustrates the state graph along with the corresponding probabilistic vectors for each iteration. Notably, in the graph:

**State Zero:** At the initial state (node 0), the probabilistic vector shows a value of 1 at node 1, indicating a certainty of transitioning to node 1 from node 0.
**Last Node:** Similarly, for the final state (node 2^size−1) the probabilistic vector assigns a value of 1 at node 2^size−1  indicating a certainty of transitioning to this node from the final state.
 
**Other Nodes:** For all other states, the probabilistic vectors are set to 0.5 for the neighboring nodes. This is because each node in these states has two possible connections, each contributing equally to the transition probabilities.

<img src="graph.png">

In [1]:
from classiq import *

size = 4 # Changing the number of qubits

@qfunc
def prepare_minus(x: QBit):
  """
  # Apply the Pauli-X gate to the qubit x. This flips the state of the qubit.
  # If the qubit is in the state |0>, it will be flipped to |1>, and vice versa. 
  # Apply the Hadamard gate to the qubit x. This creates a superposition state.
  # For |1>, it transforms to (1/sqrt(2)) * (|0> - |1>), which is the |-> state.
  """
  X(x)
  H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  """
    Applies a diffuser oracle in a quantum algorithm..
  """
  # Check if x is not equal to 0. If true, flip the state of the auxiliary qubit aux.
  aux^=(x!=0)


@qfunc
def zero_diffuzer(x: QNum):
  """
    Applies a zero diffuser oracle to the input quantum number.
  """    
  # Allocate a new auxiliary qubit named 'aux'.
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)


def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    """
        Executes a W-iteration for a quantum walk on a graph with 16 vertices.
    """
    prob = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]   # Changing this to 16 nodes
    if i == 0: # Add an especial case with the node is the first one
        prob[1]=1 
    elif i == 2**size-1: 
        
        prob[2**size-2] = 1 # Add an especial case with the node is the last one
    else:            
        prob[(i+1)% 16]=0.5     # I Change this to 16 to made this work 1%16 = 1  prob[1]=0.5
        prob[(i-1)% 16]=0.5     # I Change this to 16 to made this work  -1%16 = 3 prob[3]=0.5
    print(f'State={i}, prob vec ={prob}')        
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))


@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    """
    Executes the W-operator n times for a quantum walk on a graph.
    """
    for i in range(2**size):
      W_iteration(i,vertices,adjacent_vertices)


@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    """
    Applies an edge oracle in a quantum algorithm to detect edges based on vertex states.
    """
    res |= (((vertices+adjacent_vertices)%2) ==1)


@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
    """
    Performs a bitwise swap between two quantum bit arrays.
    """
    repeat(count= x.len, iteration= lambda i: SWAP(x[i],y[i]))
  

@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    """
    Applies the S-operator for a quantum algorithm to swap vertices based on an edge oracle.
    """
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
    """
    Main function to initialize quantum registers, apply quantum operations, and execute quantum algorithms.
    """
    allocate(size,vertices)
    hadamard_transform(vertices)
    allocate(size,adjacent_vertices)
    W_operator(vertices,adjacent_vertices)
    S_operator(vertices,adjacent_vertices)


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 